In [1]:
import pandas as pd
import itertools as it

# Data Source

This **Match Stats** dataset comes from the [Overwatch League Stats Lab](https://overwatchleague.com/en-us/statslab). It contains match data for the entire history of OWL, at a granularity of individual map rounds. It is updated regularly, and each update has a unique URL.

In [2]:
# URL collected on 2021-11-06, at which point the dataset was up-to-date through 2021-09-28
match_stats_file_url = 'https://assets.blz-contentstack.com/v3/assets/blt321317473c90505c/blt4c7ee43fcc7a63c2/61537dcd1bb8c23cf8bbde70/match_map_stats.zip'

# Raw Data Augmentation
The dataset lacks information on map type, and the full map names are not the most code-friendly, so we augment the raw dataset with two derived fields `map_type` and `map_index`. We also correct the spelling of "percent" in the column names.

The `map_type` just involves adding a literal mapping. If we end up wanting to do more re-use in the future, when there might be more maps and even map types, we could get this info from the player datasets, which do include the map type.

In [3]:
def first_word(phrase):
    return phrase.split(' ')[0]

special_index_names = {
    'Blizzard World': 'blizzworld',
    "King's Row": 'kr',
    'Route 66': 'r66',
    'Temple of Anubis': 'anubis',
    'Watchpoint: Gibraltar': 'gibraltar',
}

def create_index_name(map_name):
    return special_index_names.get(map_name, first_word(map_name).lower())

map_types = {
    'blizzworld': 'hybrid',
    'busan': 'control',
    'dorado': 'escort',
    'eichenwalde': 'hybrid',
    'hanamura': 'assault',
    'havana': 'escort',
    'hollywood': 'hybrid',
    'horizon': 'assault',
    'ilios': 'control',
    'junkertown': 'escort',
    'kr': 'hybrid',
    'lijiang': 'control',
    'nepal': 'control',
    'numbani': 'hybrid',
    'oasis': 'control',
    'paris': 'assault',
    'rialto': 'escort',
    'r66': 'escort',
    'anubis': 'assault',
    'volskaya': 'assault',
    'gibraltar': 'escort',
}

In [4]:
df_raw = pd.read_csv(match_stats_file_url).rename(
        columns=lambda c: c.replace('perecent', 'percent'),
    ).rename(
        columns={'map_round': 'round_number'}
    ).assign(
        map_index=lambda df: df.map_name.map(create_index_name),
        map_type=lambda df: df.map_index.map(map_types)
    )

# Data Cleaning

There are some small errors in the dataset, which we attempt to clean up and normalize.

## Game Number

I noticed that a few matches had errors in the game number sequence. AFAICT, the map names are all correct, but they are not necessarily unique within a match. (Not sure if this is possible anymore under the most recent tournament match rules, but it has been allowed in the past when the maps are selected by one of the competing teams for the same map to occur multiple times in the series.) There are no instances of matches with a repeated map that also have an error with the game number sequence, so we can use `(map_name, game_number)` to uniquely identify a single game within a match.

In [5]:
def get_map_order(match_group):
    return [
        (current_map, game_number) for (current_map, next_map, game_number) 
        in it.zip_longest(match_group.map_index, match_group.map_index[1:], match_group.game_number)
        if current_map != next_map
    ]

def create_corrected_game_number_lookup(match_group):
    game_keys = get_map_order(match_group)
    return { game_key: index + 1 for (index, game_key) in enumerate(game_keys) }

def correct_match_game_numbers(match_group):
    game_number_lookup = create_corrected_game_number_lookup(match_group)
    return match_group.assign(
        game_number=match_group.apply(
            lambda row: game_number_lookup[
                    (row.map_index, row.game_number)
                ], axis='columns'
            )
        )

def correct_game_numbers(rounds_df):
    return rounds_df.groupby('match_id').apply(correct_match_game_numbers).reset_index(drop=True)


## Map Round Numbers

There are several instances where the round numbers don't form the expected sequence for a game, even after the correction of the game numbers. Inspecting the instances where the largest map round number does not match the number of map round records reveals two separate issues.

* Rounds in hybrid map games in Season 1 Stage 1 are numbered with odd numbers only.
* There are a handful of instances with multiple records for a single round. Possibly due to a pause/round restart?


In [6]:
df_raw.groupby(
        ['match_id', 'game_number', 'map_index']
    ).round_number.agg(highest='max', round_count=len).query('highest != round_count').index

MultiIndex([(10223, 4,     'numbani'),
            (10224, 4, 'eichenwalde'),
            (10225, 4,     'numbani'),
            (10226, 4,     'numbani'),
            (10227, 4, 'eichenwalde'),
            (10228, 4,     'numbani'),
            (10229, 4,     'numbani'),
            (10230, 4, 'eichenwalde'),
            (10231, 4,     'numbani'),
            (10232, 4,     'numbani'),
            (10233, 4,     'numbani'),
            (10234, 4,     'numbani'),
            (10235, 4, 'eichenwalde'),
            (10236, 4, 'eichenwalde'),
            (10237, 4,     'numbani'),
            (10238, 4, 'eichenwalde'),
            (10239, 4,     'numbani'),
            (10240, 4,     'numbani'),
            (10241, 4,     'numbani'),
            (10242, 4,     'numbani'),
            (10243, 4,     'numbani'),
            (10244, 4, 'eichenwalde'),
            (10245, 4, 'eichenwalde'),
            (10246, 4, 'eichenwalde'),
            (10247, 1, 'eichenwalde'),
            (10248, 1,   

In [7]:
round_id = ['match_id', 'game_number', 'map_name', 'round_number']
print(df_raw.query('match_id == 10223 and map_index == "numbani"')[round_id]) # odd round numbers
print(df_raw.query('match_id == 34491 and map_index == "lijiang"')[round_id]) # duplicate round records

    match_id  game_number map_name  round_number
9      10223            4  Numbani             1
10     10223            4  Numbani             3
      match_id  game_number       map_name  round_number
7341     34491            1  Lijiang Tower             1
7342     34491            1  Lijiang Tower             1
7343     34491            1  Lijiang Tower             2
7344     34491            1  Lijiang Tower             3


In [8]:
def correct_round_numbers(rounds_df):
    return rounds_df.pipe(drop_duplicate_round_records).pipe(correct_round_number_sequences)

def drop_duplicate_round_records(rounds_df):
    return rounds_df.groupby(['match_id', 'game_number', 'map_index', 'round_number']).first().reset_index()

def correct_round_number_sequences(rounds_df):
    return rounds_df.groupby(
            ['match_id', 'game_number', 'map_index']
        ).apply(
            lambda game_group: game_group.sort_values('round_number').assign(round_number=range(1, len(game_group) + 1))
        # I don't know why DataFrameGroupBy methods that return a DataFrame seem to be inconsistent on whether
        # the group index columns are retained as columns or not.
        ).reset_index(drop=True)

## Results

In [9]:
df = df_raw.pipe(correct_game_numbers).pipe(correct_round_numbers)
df

,match_id,game_number,map_index,round_number,round_start_time,round_end_time,stage,match_winner,map_winner,map_loser,...,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_percent,defender_control_percent,attacker_round_end_score,defender_round_end_score,map_type
0,10223,1,dorado,1,2018-01-11 00:12:07,2018-01-11 00:20:07,Overwatch League - Stage 1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,...,San Francisco Shock,75.615051,0.000000,0.000000,240.000000,NaN,NaN,2,0,escort
1,10223,1,dorado,2,2018-01-11 00:22:05,2018-01-11 00:27:59,Overwatch League - Stage 1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,...,San Francisco Shock,75.649597,75.615051,125.750572,0.000000,NaN,NaN,3,2,escort
2,10223,2,anubis,1,2018-01-11 00:34:39,2018-01-11 00:38:29,Overwatch League - Stage 1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,...,San Francisco Shock,0.000000,0.000000,250.492004,240.000000,NaN,NaN,2,0,assault
3,10223,2,anubis,2,2018-01-11 00:40:27,2018-01-11 00:44:41,Overwatch League - Stage 1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,...,San Francisco Shock,0.000000,0.000000,225.789032,250.492004,NaN,NaN,2,2,assault
4,10223,2,anubis,3,2018-01-11 00:46:09,2018-01-11 00:49:48,Overwatch League - Stage 1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,...,San Francisco Shock,0.000000,0.000000,36.396057,250.492004,NaN,NaN,4,2,assault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11227,37448,1,nepal,2,2021-09-22 23:14:02,2021-09-22 23:17:32,OWL 2021,Atlanta Reign,Atlanta Reign,Washington Justice,...,Atlanta Reign,0.000000,0.000000,0.000000,0.000000,41.0,100.0,0,2,control
11228,37448,2,anubis,1,2021-09-22 23:25:49,2021-09-22 23:29:13,OWL 2021,Atlanta Reign,Atlanta Reign,Washington Justice,...,Atlanta Reign,0.000000,0.000000,216.344025,0.000000,NaN,NaN,2,0,assault
11229,37448,2,anubis,2,2021-09-22 23:30:44,2021-09-22 23:34:51,OWL 2021,Atlanta Reign,Atlanta Reign,Washington Justice,...,Atlanta Reign,0.000000,0.000000,0.000000,216.344025,NaN,NaN,0,2,assault
11230,37448,3,kr,1,2021-09-22 23:43:14,2021-09-22 23:48:53,OWL 2021,Atlanta Reign,Atlanta Reign,Washington Justice,...,Atlanta Reign,70.281013,0.000000,140.901001,0.000000,NaN,NaN,3,0,hybrid


### Check Game and Round Numbers

In [10]:
first_rounds = df.groupby(['match_id', 'game_number']).first().reset_index()

print('First round is always 1?:', (first_rounds.round_number == 1).all())
print(
    'Game numbers are monotonic increasing?:', 
    first_rounds.groupby('match_id').game_number.is_monotonic_increasing.all(),
)
print(
    'Round numbers are monotonic increasing?:', 
    df.groupby(['match_id', 'game_number']).round_number.is_monotonic_increasing.all(),
)

First round is always 1?: True
Game numbers are monotonic increasing?: True
Round numbers are monotonic increasing?: True


### Output

In [11]:
df.to_csv('cleaned_match_map_stats.csv', index=False)